In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/GAN/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/GAN


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math
import time


from make_model import make_discriminator_model,make_generator_model
from DP import differential_privacy
from help_function import flatten,reshape

In [3]:
#Parameters for federated learning
whether_warm_up = False
warm_up_epochs = 1
warm_up_batch_size = 256
warm_up_learning_rate = 0.001
warm_up_data_size = 3000

Train_round = 100
Client_learning_rate = 0.001
Client_batch_size = 32



Parameter_shape = []
Parameter_number = 0
Total_data_amount = 0

#Parameters for differential privicy
Whether_differential_privacy = True
Theta_d = 1
Theta_u = 0.1
Gamma = 0.001
Tua = 0.0001
Privacy_budget_per_para = 50
#Max_select_number = 10000

#Parameters for GAN

#Parameters for test


In [4]:
#Clients's behavior
class Client:
  def __init__(self,id):
    self.model = make_discriminator_model()
    self.model.compile(optimizer=keras.optimizers.Adam(learning_rate=Client_learning_rate),
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])
    server_Weight,largest_stat_index = server.send(1)
    self.model.set_weights(reshape(server_Weight,Parameter_shape))
    self.data = None
    self.labels = None
    self.data_amount = 0
    self.id = id

    self.malicious = False
    self.attack_label = None
  def training_step(self):
    Weight1 = self.model.get_weights()
    Weight2_flattened,largest_stat_index = server.send(Theta_d)
    #print(Weight2_flattened,largest_stat_index)
    Weight1_flattened = flatten(Weight1)
    Weight1_flattened[largest_stat_index] = Weight2_flattened[largest_stat_index]
    
    Weight1 = reshape(Weight1_flattened,Parameter_shape)
    #print(Weight1_flattened)
    if(self.malicious==True):
      Attack(Client_data[self.id],Client_labels[self.id],1,Weight1,self.attack_label)
    self.model.set_weights(Weight1)
    self.model.fit(self.data,self.labels,validation_split=0,epochs=1,batch_size=Client_batch_size,verbose=1)
    Gradient = np.array(self.model.get_weights()) - np.array(Weight1)
    start = time.time()
    if Whether_differential_privacy == True:
      Gradient = differential_privacy(Gradient,Privacy_budget_per_para,Gamma,Theta_u,Tua,Parameter_shape)
    print("Time spend on DP:",time.time()-start)
    return self.data_amount,Gradient

In [5]:
# Server's behavior
class Server:
  def __init__(self):
    self.model = make_discriminator_model()
    self.model.compile(optimizer=keras.optimizers.Adam(learning_rate=warm_up_learning_rate),
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])
    self.warm_up_data = None
    self.warm_up_labels = None
    self.warm_up_data_amount = 0
    self.stat = []
    for i in range(Parameter_number):
      self.stat.append([i,0])

  def warm_up(self):
    assert self.warm_up_data_amount != 0,"There is no data for warm up!"
    self.model.fit(self.warm_up_data,self.warm_up_labels,validation_split=0,epochs=warm_up_epochs,batch_size=warm_up_batch_size,verbose=0)

  #A participant uploads gradients
  def new_gradient(self,Gradient):
    Weight = self.model.get_weights()
    Weight += Gradient
    self.model.set_weights(Weight)
    
  #A participant downloads parameters.
  def send(self,theta):
    largest_stat_index = [i[0] for i in sorted(self.stat,key = lambda x:(x[1],x[0]),reverse=True)[0:math.floor(theta*Parameter_number)]]
    Weight=self.model.get_weights()
    Weight_flattened = flatten(Weight)

    selected_Weight = np.zeros(Parameter_number)
    for i in largest_stat_index:
      selected_Weight[i] = Weight_flattened[i]
    return selected_Weight,largest_stat_index

In [6]:
#Federated learning preparation
#get Parameter number and shape
test_model = make_discriminator_model()
test_weight = test_model.get_weights()
for w in test_weight:
  shape = w.shape
  p_num = 1
  for i in shape:
    p_num = p_num*i
  Parameter_shape.append((p_num,w.shape))
  Parameter_number = Parameter_number + p_num 
del test_model,test_weight

#load data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5   # Normalization
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')
test_images = (test_images - 127.5) / 127.5   # Normalization
Total_data_amount = len(train_labels)
###shuffle the data to get the warm up data
state = np.random.get_state()
np.random.shuffle(train_images)
np.random.set_state(state)
np.random.shuffle(train_labels)
warm_up_data = train_images[0:warm_up_data_size]
warm_up_labels = train_labels[0:warm_up_data_size]

#init server and clients
server = Server()
server.warm_up_data = warm_up_data
server.warm_up_labels = warm_up_labels
server.warm_up_data_amount = warm_up_data_size
if whether_warm_up == True:
  server.warm_up()
clients = []
for i in range(10):
  client = Client(i)
  client.data = train_images[train_labels==i]
  client.labels = train_labels[train_labels==i]
  client.data_amount = len(client.data)

  state = np.random.get_state()
  np.random.shuffle(client.data)
  np.random.set_state(state)
  np.random.shuffle(client.labels)
  clients.append(client)
del train_images,train_labels

In [7]:
total_start = time.time()
for i in range(Train_round):
  start = time.time()
  # Round robin
  for j in range(10):
    data_amount,Gradient = clients[j].training_step()
    Gradient = np.array(Gradient) * data_amount/Total_data_amount
    
    server.new_gradient(Gradient)
  end = time.time()
  print("Time for round:",i,"is ",end-start)
  test_loss, test_acc = server.model.evaluate(test_images,  test_labels, verbose=0)
  print("test_loss:",test_loss,"test_acc:",test_acc)
print("total time:",time.time()-total_start) 

186/186 [==============================] - 3s 4ms/step - loss: 0.1214 - accuracy: 0.9689


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Time spend on DP: 1.7948102951049805


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


211/211 [==============================] - 1s 4ms/step - loss: 0.0814 - accuracy: 0.9842
Time spend on DP: 1.7479076385498047
187/187 [==============================] - 1s 4ms/step - loss: 0.1016 - accuracy: 0.9691
Time spend on DP: 1.7692058086395264
192/192 [==============================] - 1s 4ms/step - loss: 0.1007 - accuracy: 0.9697
Time spend on DP: 1.7601816654205322
183/183 [==============================] - 1s 4ms/step - loss: 0.1054 - accuracy: 0.9705
Time spend on DP: 1.752549409866333
170/170 [==============================] - 1s 4ms/step - loss: 0.1124 - accuracy: 0.9676
Time spend on DP: 1.7579140663146973
185/185 [==============================] - 1s 4ms/step - loss: 0.1032 - accuracy: 0.9688
Time spend on DP: 1.8133058547973633
196/196 [==============================] - 1s 4ms/step - loss: 0.0916 - accuracy: 0.9702
Time spend on DP: 1.7778868675231934
183/183 [==============================] - 1s 4ms/step - loss: 0.1177 - accuracy: 0.9685
Time spend on DP: 1.7847156524